In [0]:
from sklearn.datasets import load_iris

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [0]:
iris = load_iris()

In [0]:
iris = load_iris()
iris_X, iris_y = iris.data[:-1,:], iris.target[:-1]
iris_y= pd.get_dummies(iris_y).values
trainX, testX, trainY, testY = train_test_split(iris_X, iris_y, test_size=0.33, random_state=42)

In [29]:
print(trainX.shape)
print(trainY.shape)

(99, 4)
(99, 3)


In [25]:
trainX.dtype

dtype('float64')

In [26]:
trainY.dtype

dtype('uint8')

In [0]:
numFeatures = trainX.shape[1]
numLabels = trainY.shape[1]

In [12]:
print(numFeatures)
print(numLabels)

4
3


In [0]:
# 'None' means TensorFlow shouldn't expect a fixed number in that dimension
X = tf.placeholder(tf.float32, [None, numFeatures]) # Iris has 4 features, so X is a tensor to hold our data.
yGold = tf.placeholder(tf.float32, [None, numLabels]) # This will be our correct answers matrix for 3 classes

In [30]:
X

<tf.Tensor 'Placeholder:0' shape=(?, 4) dtype=float32>

In [31]:
X.shape

TensorShape([Dimension(None), Dimension(4)])

In [0]:
#Randomly sample from a normal distribution with standard deviation .01

weights = tf.Variable(tf.random_normal([numFeatures,numLabels],
                                       mean=0,
                                       stddev=0.01,
                                       name="weights"))

bias = tf.Variable(tf.random_normal([1,numLabels],
                                    mean=0,
                                    stddev=0.01,
                                    name="bias"))

In [27]:
weights.dtype

tf.float32_ref

In [28]:
bias.dtype

tf.float32_ref

In [15]:
weights

<tf.Variable 'Variable:0' shape=(4, 3) dtype=float32_ref>

In [17]:
s = tf.Session()
init = tf.global_variables_initializer()
s.run(init)
s.run(weights)

array([[-0.00172651, -0.00297478,  0.00280098],
       [-0.00653906,  0.00534655, -0.00308367],
       [ 0.00196363,  0.00394518, -0.01274727],
       [ 0.0130916 ,  0.01458155,  0.00267545]], dtype=float32)

In [0]:
s.close()

In [0]:
# Three-component breakdown of the Logistic Regression equation.
# Note that these feed into each other.
apply_weights_OP = tf.matmul(X, weights, name="apply_weights")
add_bias_OP = tf.add(apply_weights_OP, bias, name="add_bias") 
activation_OP = tf.nn.sigmoid(add_bias_OP, name="activation")

In [0]:
# Number of Epochs in our training
numEpochs = 700

# Defining our learning rate iterations (decay)
learningRate = tf.train.exponential_decay(learning_rate=0.0008,
                                          global_step= 1,
                                          decay_steps=trainX.shape[0],
                                          decay_rate= 0.95,
                                          staircase=True)

In [32]:
learningRate

<tf.Tensor 'ExponentialDecay:0' shape=() dtype=float32>

In [34]:
r = tf.Session()
r.run(learningRate)

0.0008

In [35]:
learningRate.shape

TensorShape([])

In [0]:
r.close()

In [0]:
#Defining our cost function - Squared Mean Error
cost_OP = tf.nn.l2_loss(activation_OP-yGold, name="squared_error_cost")

#Defining our Gradient Descent
training_OP = tf.train.GradientDescentOptimizer(learningRate).minimize(cost_OP)

In [0]:
# Create a tensorflow session
sess = tf.Session()

# Initialize our weights and biases variables.
init_OP = tf.global_variables_initializer()

# Initialize all tensorflow variables
sess.run(init_OP)

In [0]:
# argmax(activation_OP, 1) returns the label with the most probability
# argmax(yGold, 1) is the correct label
correct_predictions_OP = tf.equal(tf.argmax(activation_OP,1),tf.argmax(yGold,1))

# If every false prediction is 0 and every true prediction is 1, the average returns us the accuracy
accuracy_OP = tf.reduce_mean(tf.cast(correct_predictions_OP, "float"))

# Summary op for regression output
activation_summary_OP = tf.summary.histogram("output", activation_OP)

# Summary op for accuracy
accuracy_summary_OP = tf.summary.scalar("accuracy", accuracy_OP)

# Summary op for cost
cost_summary_OP = tf.summary.scalar("cost", cost_OP)

# Summary ops to check how variables (W, b) are updating after each iteration
weightSummary = tf.summary.histogram("weights", weights.eval(session=sess))
biasSummary = tf.summary.histogram("biases", bias.eval(session=sess))

# Merge all summaries
merged = tf.summary.merge([activation_summary_OP, accuracy_summary_OP, cost_summary_OP, weightSummary, biasSummary])

# Summary writer
writer = tf.summary.FileWriter("summary_logs", sess.graph)

In [24]:
# Initialize reporting variables
cost = 0
diff = 1
epoch_values = []
accuracy_values = []
cost_values = []

# Training epochs
for i in range(numEpochs):
    if i > 1 and diff < .0001:
        print("change in cost %g; convergence."%diff)
        break
    else:
        # Run training step
        step = sess.run(training_OP, feed_dict={X: trainX, yGold: trainY})
        # Report occasional stats
        if i % 10 == 0:
            # Add epoch to epoch_values
            epoch_values.append(i)
            # Generate accuracy stats on test data
            train_accuracy, newCost = sess.run([accuracy_OP, cost_OP], feed_dict={X: trainX, yGold: trainY})
            # Add accuracy to live graphing variable
            accuracy_values.append(train_accuracy)
            # Add cost to live graphing variable
            cost_values.append(newCost)
            # Re-assign values for variables
            diff = abs(newCost - cost)
            cost = newCost

            #generate print statements
            print("step %d, training accuracy %g, cost %g, change in cost %g"%(i, train_accuracy, newCost, diff))


# How well do we perform on held-out test data?
print("final accuracy on test set: %s" %str(sess.run(accuracy_OP, 
                                                     feed_dict={X: testX, 
                                                                yGold: testY})))

step 0, training accuracy 0.333333, cost 34.3098, change in cost 34.3098
step 10, training accuracy 0.585859, cost 30.074, change in cost 4.23582
step 20, training accuracy 0.646465, cost 28.1352, change in cost 1.93882
step 30, training accuracy 0.646465, cost 26.4989, change in cost 1.63627
step 40, training accuracy 0.646465, cost 25.1393, change in cost 1.35967
step 50, training accuracy 0.646465, cost 24.0158, change in cost 1.12348
step 60, training accuracy 0.646465, cost 23.0856, change in cost 0.930134
step 70, training accuracy 0.646465, cost 22.3106, change in cost 0.775064
step 80, training accuracy 0.646465, cost 21.659, change in cost 0.651573
step 90, training accuracy 0.646465, cost 21.1058, change in cost 0.553194
step 100, training accuracy 0.656566, cost 20.6314, change in cost 0.474442
step 110, training accuracy 0.666667, cost 20.2204, change in cost 0.410944
step 120, training accuracy 0.666667, cost 19.8611, change in cost 0.359324
step 130, training accuracy 0.6